In [1]:
# switch to the project directory
%cd ..
# working directory should be ../pdi

c:\Users\admin\Desktop\research\pdi


In [2]:
import sys
import os
module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from pdi.constants import (
    PARTICLES_DICT,
    TARGET_CODES
)
from pdi.data.constants import GROUP_ID_KEY

In [4]:
import torch
from pdi.data.preparation import FeatureSetPreparation
from pdi.data.types import Split
import onnx

device = torch.device("cpu")

data_preparation = FeatureSetPreparation()
(train_loader, ) = data_preparation.prepare_dataloaders(64, 0, [Split.TRAIN])

input_data, _, data_dict = next(iter(train_loader))
gid = data_dict.get(GROUP_ID_KEY)

dummy_input = (input_data.to(device), gid.to(device))

os.makedirs(f"onnx/Proposed/", exist_ok=True)
for target_code in TARGET_CODES:
    load_path = f"models/Proposed/{PARTICLES_DICT[target_code]}.pt"
    export_path = f"onnx/Proposed/{PARTICLES_DICT[target_code]}.onnx"
    model = torch.load(load_path).to(device)
    #model_script = torch.jit.script(model)
    torch.onnx.export(model, dummy_input, export_path)

    onnx_model = onnx.load(export_path)
    onnx.checker.check_model(onnx_model)
    print(onnx.helper.printable_graph(onnx_model.graph))

graph torch_jit (
  %onnx::MatMul_0[FLOAT, 64x15x20]
) initializers (
  %emb.layers.0.bias[FLOAT, 128]
  %emb.layers.2.bias[FLOAT, 32]
  %encoder.layers.0.self_attn.in_proj_bias[FLOAT, 96]
  %encoder.layers.0.self_attn.out_proj.weight[FLOAT, 32x32]
  %encoder.layers.0.self_attn.out_proj.bias[FLOAT, 32]
  %encoder.layers.0.linear1.bias[FLOAT, 128]
  %encoder.layers.0.linear2.bias[FLOAT, 32]
  %encoder.layers.0.norm1.weight[FLOAT, 32]
  %encoder.layers.0.norm1.bias[FLOAT, 32]
  %encoder.layers.0.norm2.weight[FLOAT, 32]
  %encoder.layers.0.norm2.bias[FLOAT, 32]
  %encoder.layers.1.self_attn.in_proj_bias[FLOAT, 96]
  %encoder.layers.1.self_attn.out_proj.weight[FLOAT, 32x32]
  %encoder.layers.1.self_attn.out_proj.bias[FLOAT, 32]
  %encoder.layers.1.linear1.bias[FLOAT, 128]
  %encoder.layers.1.linear2.bias[FLOAT, 32]
  %encoder.layers.1.norm1.weight[FLOAT, 32]
  %encoder.layers.1.norm1.bias[FLOAT, 32]
  %encoder.layers.1.norm2.weight[FLOAT, 32]
  %encoder.layers.1.norm2.bias[FLOAT, 32]
  %po